# plotHCRmoments
This script imports and plots radar moments from concatenated versions (_using `concatHCRmoments`_) of the HIAPER Cloud Radar data.
<br><br>
Typically this script will take ~30 min to plot 15-min 4-panel plots for ~7 hours of flight time (run on an early 2015 MacBook Pro).
<br><br>
_Written by Dan Stechman_
<br>
_University of Illinois at Urbana-Champaign_

In [ ]:
import pyart
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import datetime
import matplotlib.dates as mdates
import warnings
import os

dt = datetime.datetime

scriptStrtT = dt.now()

warnings.filterwarnings("ignore",category=RuntimeWarning)

# np.set_printoptions(precision=3,suppress=True,threshold=10000) # Enable to control number format when printing values within notebook
# %matplotlib inline # Enable if plotting single time periods interactively - will display plot within notebook

## User Settings
This is where the most commonly modified settings reside (i.e., flight, plot start/end times, data location, save path and file type, etc.)

In [ ]:
# This variable should match the name of the parent directory containing
#    the flight data for the given mission. Used in defining file input 
#    and output names
flight = 'RF07_20180131'

# Specifiy the type of output plot you'd like
#    Currently, this script only supports making one type at a time
fourPanel_5min     = False # Plot 5-min periods spanning startT-endT
fourPanel_15min    = True # Plot 15-min periods spanning startT-endT
fourPanel_variable = False # Plot only user-defined periods of variable length (see RF02 settings below for an example)

# Define flight-specific parameters, namely the period over which to produce plots
#    # RF02 case below is commented - please read for more information
if flight == 'RF01_20180116':
    startT = '20180115_230000'
    endT = '20180116_054500'
    
if flight == 'RF02_20180119':
    # If we're only plotting between a single start and end, define only those times
    if not fourPanel_variable:
        # startT and endT should be strings of the format 'YYYYmmdd_HHMMSS'
        #    Be sure that the defined range is evenly divisible by your plot
        #    period (i.e., start and end at 00, 15, 30, or 45 min if plot period is every 15 min)
        startT = '20180119_004500'
        endT = '20180119_064500'
        
    else:
        # Change this to plot the Nth segment in the following set of arrays
        #    Currently, you'll need to run the code for each segment 
        #   (I will automate this if this is commonly used)
        segNum = 11

        # Set startT and endT pairs for every segment. The ranges between these can be 
        #    whatever you wish, though times must be rounded to minutes (SS must equal 00)
        startT_segs = ['20180119_033500','20180119_034600','20180119_035300','20180119_041400',
                       '20180119_042300','20180119_044500','20180119_050100','20180119_051200',
                      '20180119_052500','20180119_054200','20180119_055500']
        endT_segs = ['20180119_034500','20180119_035200','20180119_040200','20180119_042300',
                     '20180119_043500','20180119_050000','20180119_051100','20180119_051800',
                    '20180119_053300','20180119_054800','20180119_063200']
        
        
        startT_s = [dt.strptime(sT,'%Y%m%d_%H%M%S') for sT in startT_segs]
        endT_s = [dt.strptime(eT,'%Y%m%d_%H%M%S') for eT in endT_segs]
        dur_segs = [int((eT-sT).seconds/60) for eT,sT in zip(endT_s,startT_s)]

        startT = startT_segs[segNum-1]
        endT = endT_segs[segNum-1]
        
    
if flight == 'RF03_20180123':
    startT = '20180122_211500'
    endT = '20180123_034500'
    
if flight == 'RF04_20180124':
    startT = '20180123_233000'
    endT = '20180124_054500'
if flight == 'RF05_20180126':
    startT = '20180125_230000'
    endT = '20180126_051500'
if flight == 'RF06_20180129':
    startT = '20180128_230000'
    endT = '20180129_060000'
if flight == 'RF07_20180131':
    startT = '20180131_010000'
    endT = '20180131_073000'


# Define plot period duration and save directory name
if fourPanel_5min:
    dur = 5
    saveDir = '5min_4panel'
    
if fourPanel_15min:
    dur = 15
    saveDir = '15min_4panel'

if fourPanel_variable:
    dur = dur_segs[segNum-1] # Duration of plot period in minutes
    saveDir = 'variableSegs_4panel'

   
# Create a timedelta corresponding to out plotting period
tDelta = datetime.timedelta(minutes=dur)

# Define the path/name of our concatenated HCR data file
hcrFile = '/Volumes/SOCRATES_1/' + flight + '/HCR/' + flight + '_concat-HCR-moments.nc'

# Define the path where figures should be saved and make any required directories
figSavePath = '/Users/danstechman/GoogleDrive/School/Research/SOCRATES/UI_OU_SOCRATES_Group/SOCRATES/Plots/' + flight + '/HCR/' + saveDir + '/'
os.makedirs(figSavePath,exist_ok=True)

# What type of file do we want our plots to be saved as? (i.e., png, pdf, jpeg,...)
fType = 'png'

## Read data
Read in the concatenated HCR variables

In [ ]:
if 'dbz' not in locals(): # Used to speed up variable segment plotting - this condition can be disabled
    print('{}    Reading concatenated HCR data file for {}...'.format(dt.strftime(dt.now(),'%y/%m/%d %H:%M:%S'),flight))
    hcrData = xr.open_dataset(hcrFile)

    time1d = np.asarray(hcrData['time1d'].data,dtype='datetime64[ns]')
    time1d_rnd = (pd.to_datetime(time1d)).round('1s').values
    time2d = np.asarray(hcrData['time2d'].data,dtype='datetime64[ns]')
    gateAlt = hcrData['gateAlt2d'].data
    elev = hcrData['elevation'].data
    
    planeAlt = hcrData['planeAlt'].data

    # Determine the time indices where the HCR is pointing downward
    radDwnwrd = np.where(elev < 0.0)

    dbz = hcrData['DBZ'].data
    vel = hcrData['VEL'].data
    width = hcrData['WIDTH'].data
    ldr = hcrData['LDR'].data
    ncp = hcrData['NCP'].data
    snr = hcrData['SNR'].data

    # Adjust radial velocities so negative values are always downward
    vel[radDwnwrd,:] *= -1

## Mask Variables
Mask variables to be plotted based on a number of (adjustable) criteria.

In [ ]:
if 'dbz_masked_ncp' not in locals(): # Used to speed up variable segment plotting - this condition can be disabled
    print('{}    Masking radar moments...'.format(dt.strftime(dt.now(),'%y/%m/%d %H:%M:%S')))
    dbz_masked_ncp = np.ma.masked_where((ncp < 0.2)|(gateAlt < 0)|np.isnan(dbz)|np.isinf(dbz),dbz)
    vel_masked_ncp = np.ma.masked_where((ncp < 0.2)|(gateAlt < 0)|np.isnan(dbz)|np.isinf(dbz),vel)
    width_masked_ncp = np.ma.masked_where((ncp < 0.2)|(gateAlt < 0)|np.isnan(dbz)|np.isinf(dbz),width)
    ldr_masked_ncp = np.ma.masked_where((ncp < 0.2)|(gateAlt < 0)|np.isnan(dbz)|np.isinf(dbz),ldr)

## Data Index ID
Determine which data indices will provide data between the user-defined start and end times

In [ ]:
# Convert the start and end time strings into datetimes
startT_dt = dt.strptime(startT,'%Y%m%d_%H%M%S')
endT_dt = dt.strptime(endT,'%Y%m%d_%H%M%S')

# Find indices of the time variable most closely matching startT_dt and endT_dt
tMatchStrt = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - startT_dt))
startT_ix = np.squeeze(np.where(time1d_rnd == tMatchStrt))[0] # If multiple matches, just use the first one (earliest)
tMatchEnd = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - endT_dt))
endT_ix = np.squeeze(np.where(time1d_rnd == tMatchEnd))[-1] # If multiple matches, just use the last one (latest)

# Get the actual start/end datetimes (from the time variable, not the user-defined start/end)
strtDT_rnd = pd.to_datetime(time1d_rnd[startT_ix])
endDT_rnd = pd.to_datetime(time1d_rnd[endT_ix])

## Create plots

In [ ]:
print('{}    Starting plot creation...'.format(dt.strftime(dt.now(),'%y/%m/%d %H:%M:%S')))

# Initialize the plotting frame start and end times
tmpStrtDT = strtDT_rnd
tmpEndDT = strtDT_rnd + tDelta

# Loop through our data until the end time of a given plotting frame exceeds the
#   end time defined by the user
#   **The extra minute timedelta is a fudge factor needed due to a currently unidentified bug
while tmpEndDT <= (endDT_rnd + datetime.timedelta(minutes=1)):
    
    # Define plot title and filename strings depending on the time range of the plot,
    #    and whether the plot frame spans two days (i.e., don't print the date twice, 
    #    if it doesn't actually change)
    if tmpStrtDT.day == tmpEndDT.day:
        titleDTstr = '{} - {}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H:%M:%S'),
                                         dt.strftime(tmpEndDT,'%H:%M:%S'))
        if tDelta <= datetime.timedelta(minutes=5):
            saveDTstr = '{}_{}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M%S'),
                                             dt.strftime(tmpEndDT,'%H%M%S'))
        else:
            saveDTstr = '{}_{}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M'),
                                             dt.strftime(tmpEndDT,'%H%M'))
    else:
        titleDTstr = '{}-{} - {}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H:%M:%S'),
                                         dt.strftime(tmpEndDT,'%Y%m%d'),dt.strftime(tmpEndDT,'%H:%M:%S'))
        if tDelta <= datetime.timedelta(minutes=5):
            saveDTstr = '{}_{}-{}_{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M'),
                                             dt.strftime(tmpEndDT,'%Y%m%d'),dt.strftime(tmpEndDT,'%H%M%S'))
        else:
            saveDTstr = '{}_{}-{}_{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M'),
                                             dt.strftime(tmpEndDT,'%Y%m%d'),dt.strftime(tmpEndDT,'%H%M'))
    
    print('\tPlotting {}'.format(titleDTstr))
    
    # If our plotting period is longer than 5 minutes, don't show seconds in the xtick labels
    if tDelta <= datetime.timedelta(minutes=5):
        xtick_formatter = mdates.DateFormatter(fmt='%H:%M:%S')
    else:
        xtick_formatter = mdates.DateFormatter(fmt='%H:%M')
    
    
    # Find start and end indices most closely matching current plotting frame bounds
    tMatchStrt = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - tmpStrtDT))
    tmpStIx = np.squeeze(np.where(time1d_rnd == tMatchStrt))[0]
    tMatchEnd = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - tmpEndDT))
    tmpEndIx = np.squeeze(np.where(time1d_rnd == tMatchEnd))[-1]

    
    # Initialize figure with 4 subplots, sharing x and y axes
    # Figure size should *roughly* yield a 1:1 ratio for the plot dimensions, but this is
    #    entirely dependent on aircraft speed at any given time
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True, sharey=True,figsize=(16*(dur/5.),28))
    axFsz = 32
    caxFsz = 28
    ttlFsz = 36
    tkFsz = 24
    ctkFsz = 20
    caxShrnk = 0.9
    
    # Optionally define strings to append to the title and/or filename for special cases
    if fourPanel_variable:
        titleAppnd = ' - Seg #' + str(segNum)
        saveAppnd = '_Seg' + str(segNum)
    else:
        titleAppnd = ''
        saveAppnd = ''

        
    # Plot reflectivity and create our figure title
    im1 = ax1.pcolormesh(time2d[tmpStIx:tmpEndIx,:],gateAlt[tmpStIx:tmpEndIx,:]/1000,dbz_masked_ncp[tmpStIx:tmpEndIx,:],
                         vmin=-40,vmax=16,cmap=pyart.graph.cm.LangRainbow12)
    ax1.plot(time2d[tmpStIx:tmpEndIx,:],planeAlt[tmpStIx:tmpEndIx]/1000,
             'k-',linewidth=6)
    ax1.set_ylim([0,7])
    cax1 = fig.colorbar(im1,ax=ax1,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax1.set_label('Reflectivity (dBZ)',fontsize=caxFsz)
    cax1.ax.set_yticklabels(cax1.ax.get_yticklabels(), fontsize=ctkFsz)
    ax1.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax1.set_title('SOCRATES - {} UTC{}'.format(titleDTstr,titleAppnd),fontsize=ttlFsz) 
    ax1.grid()


    # Plot radial velocity and label the y-axis
    im2 = ax2.pcolormesh(time2d[tmpStIx:tmpEndIx,:],gateAlt[tmpStIx:tmpEndIx,:]/1000,vel_masked_ncp[tmpStIx:tmpEndIx,:],
                         vmin=-5,vmax=5,cmap=pyart.graph.cm.Wild25_r)
    ax2.plot(time2d[tmpStIx:tmpEndIx,:],planeAlt[tmpStIx:tmpEndIx]/1000,
             'k-',linewidth=6)
    ax2.set_ylim([0,7])
    cax2 = fig.colorbar(im2,ax=ax2,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax2.set_label('Radial Velocity (m/s)',fontsize=caxFsz)
    cax2.ax.set_yticklabels(cax2.ax.get_yticklabels(), fontsize=ctkFsz)
    ax2.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax2.set_ylabel('Altitude (km)',fontsize=axFsz)
    ax2.grid()


    # Plot LDR
    im3 = ax3.pcolormesh(time2d[tmpStIx:tmpEndIx,:],gateAlt[tmpStIx:tmpEndIx,:]/1000,ldr_masked_ncp[tmpStIx:tmpEndIx,:],
                         vmin=-40,vmax=0,cmap=pyart.graph.cm.NWSRef)
    ax3.plot(time2d[tmpStIx:tmpEndIx,:],planeAlt[tmpStIx:tmpEndIx]/1000,
             'k-',linewidth=6)
    ax3.set_ylim([0,7])
    cax3 = fig.colorbar(im3,ax=ax3,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax3.ax.set_yticklabels(cax3.ax.get_yticklabels(), fontsize=ctkFsz)
    cax3.set_label('LDR (dB)',fontsize=caxFsz)
    ax3.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax3.grid()


    # Plot spectral width and label the x-axis (time)
    im4 = ax4.pcolormesh(time2d[tmpStIx:tmpEndIx,:],gateAlt[tmpStIx:tmpEndIx,:]/1000,width_masked_ncp[tmpStIx:tmpEndIx,:],
                         vmin=0,vmax=3,cmap=pyart.graph.cm.RefDiff)
    ax4.plot(time2d[tmpStIx:tmpEndIx,:],planeAlt[tmpStIx:tmpEndIx]/1000,
             'k-',linewidth=6)
    cax4 = fig.colorbar(im4,ax=ax4,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax4.set_label('Spectral Width (m/s)',fontsize=caxFsz,)
    cax4.ax.set_yticklabels(cax4.ax.get_yticklabels(), fontsize=ctkFsz)
    ax4.set_xlabel('Time (UTC)',fontsize=axFsz)
    ax4.xaxis.set_major_locator(mdates.MinuteLocator())
    ax4.xaxis.set_major_formatter(xtick_formatter)
    ax4.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax4.grid()


    # Clean up the date format a bit further and remove extra whitespace between subplots
    fig.autofmt_xdate()
    fig.subplots_adjust(hspace=0.08)


    # Save the output figure
    saveStr = '{}HCR-moments_{}{}.{}'.format(figSavePath,saveDTstr,saveAppnd,fType)
    fig.savefig(saveStr,bbox_inches='tight')

    # Set the start and end times for the next plot
    tmpStrtDT = tmpEndDT
    tmpEndDT += tDelta

print('\nTotal script run time: {}'.format(dt.now() - scriptStrtT))